# Prévision de courbes de charge de consommation électrique

Dans ce TP nous allons travailler sur un jeu de données qui représente la consommation électrique d'un site sur une longue durée avec une résolution assez fine. L’objectif est de construire un modèle afin de **prévoir la courbe de charge** de la consommation électrique future.

Les données forment une **série temporelle**. On observe la courbe de charge à intervalle régulier (toutes les 10 minutes) sur une longue durée  et on souhaite prédire le futur.

![Série temporelle](img/timeSeries1.jpeg)

Plus précisément, nous souhaitons un modèle qui prédit la courbe de charge électrique pour le jour suivant par rapport à l'historique des données. Par exemple, si nous sommes le 09/10/2012 à 13:30, il s'agit de prévoir la courbe de charge de consommation sur les prochaines 24 heures (toutes les 10 minutes, c'est-à-dire 144 valeurs).


En statistique il existe des modèles puissants pour l'études des séries temporelles et il est également possible d'utiliser une approche deep learning pour aborder ce problème. En revanche, vu que nous n'avons pas le temps dans ce cours d'étudier ces modèles, nous commencerons par explorer l'utilisation d'un simple **modèle linéaire** et nous verrons ensuite une première applications des forêts aléatoires.



L’atelier est scindé en deux étapes :

* **Étape 1**. **Pré-traitement des données.**  
Cette étape contient :

    - L’import et l’audit des données : vérification rapide du contenu des fichiers de travail. Permet d’avoir une idée des méthodologies possibles à utiliser. 

    - Mise au format des données. En effet, pour étudier cette série temporelle, il va falloir découper les données en périodes (d'une durée à fixer) dont on va observer des réplicats (dans le temps).
    
* **Étape 2**. **Modélisation : apprentissage et test.**  
Plusieurs modélisations sont proposées :

    - Approche naïve
    - Régression linéaire
    - Random Forest
    - Gradient Boosting


## Préparation et exploration des données

Les données à disposition sont contenues dans le fichier  `Courbes_Charge08.csv` à charger et qui contient 

   - la **consommation d'éléctricité** relevée **toutes les 10 minutes sur le site ID08**
   - la **température** sur le site ID08 relevée physiquement toutes les 3 heures. Les données sur les temps intermédiaires ont été complétées par interpolation linéaire. 

Avant d'importer le fichier, on peut  afficher les premières lignes avec la commande suivante :

In [ ]:
%%bash
head -10 'data/Courbes_Charge08.csv'

Si la commande précédente ne fonctionne pas sur votre OS :

In [ ]:
file = open('data/Courbes_Charge08.csv', 'r')
for i, line in zip(range(5), file):
   print(i, line.strip())

Notez qu'on ne sait pas si les dates sont codées en jour/mois/année ou mois/jour/année. On peut le voir en affichant une ligne un peu plus loin : 

In [ ]:
for i, line in enumerate(file):
    if i == 4000:
        print(i, line)
        break

Là on constate qu'il s'agit d'un codage jour/mois/année.

### Exercice 1


Importez les données dans un dataframe (de la bibliothèque `pandas`) nommé `df_raw` :

- L'option `parse_dates` de la fonction `read_csv` de pandas vous permet de lire correctement la variable de la date. 
- Faites attention au format de la température. 
- On transformera les noms de variables en minuscules (usage python). 

### Réponse

In [ ]:
import os
import numpy as np
import pandas as pd

# Votre chemin. Par defaut celui du notebook
# path_data = '.'
path_data = 'data/'
    
# Lecture du fichier
df_raw = pd.read_csv(os.path.join(path_data, 'Courbes_Charge08.csv'), 
                 sep=';', 
                 parse_dates=['DATE_LOCAL'],
                 decimal=',',  # les décimales sont codées par une virgule
                 dayfirst=True, # the day appears before month
                 )

# L'usage en Python est de mettre les noms de variables en minuscule 
df_raw.columns = [x.lower() for x in df_raw.columns]

In [ ]:
df_raw.head()

In [ ]:
df_raw.tail()

In [ ]:
df_raw['date_local'][0].weekday()

### Exercice 2

Nous allons créer, à partir du dataframe de départ `df_raw`, un deuxième dataframe `df` sur lequel nous allons travailler.

- Répartir l'information contenue dans la colonne `date_local` de `df_raw` en plusieurs colonnes de sorte à afficher séparement la date, le numéro de semaine, le numéro de jour, le jour de la semaine (0 pour lundi, 1 pour mardi, ...) et l'heure. Nous allons travailler uniquement sur la charge, il faut donc également supprimer la colonne des températures.

- Vérifier qu'il n'y a pas de données manquantes.

- Travailler sur des semaines complètes, qui commencent un lundi.


### Réponse

In [ ]:
from datetime import datetime
# L'ordinal est le nombres jours qui se sont écoulés à partir du 1er janvier de l'année 0001
print(datetime.strptime('0001-01-01', '%Y-%m-%d').toordinal())

first_day = df_raw["date_local"][0].toordinal()
print(first_day)

In [ ]:
first_day = df_raw["date_local"][0].toordinal()
first_day_wd = df_raw["date_local"][0].weekday()
df = pd.concat(
    [ 
        df_raw["date_local"].transform({
            "date": lambda d: d, # on garde le point temporel d'origine 
            "week": lambda d: (d.toordinal() - (first_day - first_day_wd)) // 7,
            "day": lambda d: d.toordinal() - first_day,
            "weekday": lambda d: d.weekday(),
            "time": lambda d: d.strftime("%H:%M"),
        }),
        df_raw["charge_id08"]
    ], 
    axis=1)

In [ ]:
df.loc[144*np.arange(15)]

In [ ]:
# iloc : position le long de l'index
# loc : basé sur le label
df.index  = np.arange(10, 93312+10)
df.iloc[0:10].equals(df.loc[np.arange(10,20)])

# le slicing avec loc prend le debut ET la fin du slice 
df.loc[10:1018,'week']

# loc avec condition logique
df.loc[df['week'] == 1]

df.index  = np.arange(df.shape[0])

In [ ]:
df.shape

In [ ]:
# On vérifie qu'il n'y a pas de données manquantes
print("Il y a ", df.isna().sum().sum(), "données manquantes")

# Les jours sont des jours complets
print("Nombre de jours observés : ", df.shape[0]/144)
nb_days = int(df.shape[0]/144)

# Les semaines ne sont pas complètes 
print("Nombre de semaines observées : ", df.shape[0]/(144*7))

# Les données commencent à minuit du premier jour
# et il n'y a pas d'intervals de temps manquants
# donc on pourra faire un reshape 144
print(df['time'].iloc[:144])
np.sum(np.tile(df['time'].iloc[:144],nb_days) == df['time'])

### Exercice 3

`pandas` est dit *row-major*, ce qui signifie que les éléments d'un tableau multidimensionnel sont disposés séquentiellement ligne par ligne. Ceci est cohérent avec la vision d'un dataframe comme un ensemble d'individus (ou d'observations), chacun représenté par une ligne du tableau, qui ont des caractéristiques, représentées par les colonnes. 
Dans notre exemple, pour prendre en compte la périodicité des données (qui sera de un jour ou bien d'une semaine),  il convient de transformer le dataframe pour que chaque ligne corresponde à une période.  

#### Période: 1 jour

- Créer un dataframe `df_day` avec les charges ordonnées par période d'un jour. Les colonnes de ce dataframe vont être les 144 intervals de 10 minutes qui donnent un jour. On pourra utiliser la fonction `reshape`.
- Visualiser les données de `df_day`, en affichant moyenne, min, max et déviation standard.

### Réponse

In [ ]:
# À la main
charge_matrix_by_day = df["charge_id08"].to_numpy().reshape(-1, 144)
df_day = pd.DataFrame(charge_matrix_by_day)
df_day.index.name = 'day'
df_day.columns = df['time'].iloc[:144]

# Ou bien de façon équivalente (avec multiindex directement)
#df_day = df.pivot(index=['week','weekday'], columns=['time'], values='charge_id08')

df_day

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as md
import seaborn as sns
sns.set_theme()

In [ ]:
# on se donne une date générique 01/01/2023 que l'on utilise pas dans la suite 
# c'est uniquement pour générer la séquence des 144 instants de la journée (toutes les 10 minutes)
timesteps_day = np.arange('2023-01-01', '2023-01-02', 
                      np.timedelta64(10, 'm'), dtype='datetime64')

fig, ax = plt.subplots()
ax.plot(timesteps_day, df_day.mean(axis=0), label="mean")
ax.fill_between(timesteps_day, df_day.min(axis=0), df_day.max(axis=0), alpha=0.2, label="min-max")
up = df_day.mean(axis=0) + df_day.std(axis=0)
down = df_day.mean(axis=0) - df_day.std(axis=0)
ax.fill_between(timesteps_day, down, up, alpha=0.2, label="standard deviation")
ax.legend()

ax.xaxis.set_major_formatter(md.DateFormatter("%H:%M"))

ax.set_title("Variabilité des profils de charge par jour sur tout le dataset")
fig.tight_layout()

### Exercice 4

#### Période: 1 semaine

- À l'aide de la fonction `pd.MultiIndex.from_arrays` créer un multi-index pour `df_day` qui affiche le numéro de semaine et le numéro de jour de la semaine.

- Créer un dataframe `df_week` avec les charges ordonnées par période d'une semaine à partir du dataframe `df_day` en utilisant les fonctions `stack` et `unstack`. Les colonnes de ce dataframe vont être les 144 intervals de 10 minutes qui donnent un jour, répétés 7 fois, une pour chaque jour de la semaine.

- Visualiser les données de `df_week` comme fait précédemment pour `df_day`.

### Réponse

In [ ]:
pd.MultiIndex.from_arrays([df['week'][144*np.arange(648)], df['weekday'][144*np.arange(648)]])

In [ ]:
df_day.index

In [ ]:
# On change les indexes de df_day pour prendre en compte les semaines
df_day_index = pd.MultiIndex.from_arrays([df['week'][144*np.arange(648)], df['weekday'][144*np.arange(648)]],
                                         names=('week', 'weekday'))
df_day.index = df_day_index

In [ ]:
print("Columns names", df_day.columns.names)
print("Index names ", df_day.index.names)

In [ ]:
df_day

In [ ]:
# On accède aux éléments de df_day par multi-index
print(df_day.loc[(92,4)])
# Ou bien par position
print(df_day_index.get_loc((92,4)))
print(df_day.iloc[df_day_index.get_loc((92,4))])

In [ ]:
df_day.stack().unstack([1,2]).sort_index(axis=1)

In [ ]:
# df_week est une nouvelle vue sur df_day 
df_week = df_day.stack().unstack([1,2]).sort_index(axis=1)
df_week

# Ou bien de façon équivalente
#df_week = df.pivot(index=['week', 'weekday'], columns='time', values='charge_id08')

In [ ]:
timesteps_week = np.arange('2023-01-01', '2023-01-08', 
                      np.timedelta64(10, 'm'), dtype='datetime64')

fig, ax = plt.subplots()
ax.plot(timesteps_week, df_week.mean(axis=0), label="mean")
ax.fill_between(timesteps_week, df_week.min(axis=0), df_week.max(axis=0), alpha=0.2, label="min-max")
up = df_week.mean(axis=0) + df_week.std(axis=0)
down = df_week.mean(axis=0) - df_week.std(axis=0)
ax.fill_between(timesteps_week, down, up, alpha=0.2, label="standard deviation")
ax.legend()

ax.xaxis.set_major_formatter(md.DateFormatter("%w"))

ax.set_title("Variabilité des profils de charge par semaine sur tout le dataset")
fig.tight_layout() 

## Modélisation : prédiction niveau 0

Une approche naïve consiste à utiliser comme prédicteur :
- soit la charge moyenne par heure (prise sur tous les jours de l'année via le dataframe `df_day`) 
- soit la charge moyenne par heure et par type de jour (en utilisant la périodicité hebdomadaire de `df_week`)

### Exercice 5

Créer un premier prédicteur naïf avec la moyenne par jour et la moyenne par semaine.

- Créer un predicteur naïf avec la moyenne par heure sur l'ensemble du dataframe, et afficher l'erreur quadratique et l'erreur MAPE (erreur moyenne absolue en pourcentage)
- Faire la même chose avec une période d'une semaine, en utilisant comme prédicteur la moyenne par heure et par type de jour.

*Remarque* : Nous n'allons pas séparer les données en train set et test set, il s'agit juste de fournir un prédicteur naïf à comparer avec les suivants.

### Réponse

In [ ]:
# Moyenne par heure
lazy_day = df_day.mean(axis=0)
# Différence avec la vraie valeur
lazy_day_errors = df_day - lazy_day

lazy_day_errors

In [ ]:
fig,ax = plt.subplots()
ax.plot(np.sqrt((lazy_day_errors**2).mean(axis=1)).reset_index(drop=True))

In [ ]:
# RMSE et MAPE commises chaque jour
RMSE_lazy_day = np.sqrt((lazy_day_errors**2).mean(axis=1)).reset_index(drop=True)
MAPE_lazy_day = abs(lazy_day_errors / df_day).mean(axis=1).reset_index(drop=True)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=1, nrows=2, sharex=True, layout="tight")
ax1.plot(df['day'].unique(),RMSE_lazy_day, label="RMSE (day)")  
ax1.set_title("RMSE")
ax2.plot(df['day'].unique(),MAPE_lazy_day, label="MAPE (day)")
ax2.set_title("MAPE")
for ax in (ax1, ax2): ax.legend()
plt.show()

In [ ]:
# Moyenne par heure et par jour de la semaine
lazy_week = df_week.mean(axis=0)
# Différence avec la vraie valeur
lazy_week_errors = df_week - lazy_week
lazy_week_errors

In [ ]:
# L'indice weekday est au level 0 ds les noms des colonnes
lazy_week_errors.columns.names

In [ ]:
# On revient à une structure par jour, pour calculer l'erreur 
# et comparer avec lazy_day_errors
lazy_week_errors = lazy_week_errors.stack(level = 0)
lazy_week_errors
# pareil que 
# lazy_week_errors.stack().stack().unstack([1])

In [ ]:
# RMSE et MAPE de chaque jour
RMSE_lazy_week = np.sqrt((lazy_week_errors**2).mean(axis=1)).reset_index(drop=True)
MAPE_lazy_week = abs(lazy_week_errors / df_week).mean(axis=1).reset_index(drop=True)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=1, nrows=2, sharex=True, layout="tight")
ax1.plot(df['day'].unique(),RMSE_lazy_day, label="RMSE (day)", linewidth=1)
ax1.plot(df['day'].unique(),RMSE_lazy_week, label="RMSE (week)", linewidth=1)
ax1.set_title("RMSE")
ax2.plot(df['day'].unique(),MAPE_lazy_day, label="MAPE (day)", linewidth=1)
ax2.plot(df['day'].unique(),MAPE_lazy_week, label="MAPE (week)", linewidth=1)
ax2.set_title("MAPE")
for ax in (ax1, ax2): ax.legend()
plt.show()

### Exercice 6

Faire une première analyse des résultats obtenus.

1. Une modélisation *à la journée* est-t-elle judicieuse à votre avis ?

2. Choisir de modéliser *à la semaine* plutôt qu'*à la journée* ne semble-t-il pas plus pertinent ? Pourquoi ? 

Réponse : 

1. Non ce n'est pas judicieux car les jours ont des comportements différents (typiquement le dimanche a un profil de courbe de charge très différent). 

2. Modéliser à la semaine est donc plus intéressant, puisque la périodicité des comportements est sur 7 jours. 

In [ ]:
# On vérifie également par les erreurs que l'approche par semaine marche mieux
print(RMSE_lazy_day.mean())
print(RMSE_lazy_week.mean())

print(MAPE_lazy_day.mean())
print(MAPE_lazy_week.mean())

## Modèle linéaire

Pour mettre en oeuvre un modèle linéaire, plusieurs approches sont possibles. 
### Modèle sans périodicité

Notons  $x_t$ pout $t=1,2,3,...,T$  la suite de  charges électriques observées, où l'indice $t$ fait référence à la plus ancienne observation et $x_T$ à la toute dernière.  L'interval de temps entre deux observations $x_t$ et $x_{t+1}$  est toujours de 10 minutes. 

Une première approche consiste à essayer de prédire la future consommation électrique en utilisant la consommation passée, à partir des $d$ observations qui précèdent l'instant actuel (nous appelons $d$ la profondeur de l'historique).
Nous cherchons alors une fonction $f$ telle que
\begin{equation}
x_{t+1} = f(x_{t-d+1}, x_{t-d+2}, ..., x_{t-1}, x_t) + \varepsilon_{t+1},
\tag{1}
\end{equation}
où $\varepsilon_{t+1}$ est un petit bruit aléatoire.


![Série temporelle](img/timeSeries2.jpeg)

Dans un modèle linéaire, on suppose que $f$ est une application linéaire de la forme
\begin{equation*}
f(\mathbf x) = \mathbf x^\top \beta = x_{t-d+1}\beta_1+ x_{t-d+2}\beta_2+...+ x_{t-1}\beta_{d-1}+x_{t}\beta_{d},
\tag{2}
\end{equation*}
où $\mathbf x=(x_{t-d+1}, x_{t-d+2}, ..., x_{t-1}, x_t)^\top$ et $\beta\in\mathbb R^d$ est un vecteur de paramètres inconnus à estimer à partir des données.  

*Remarque :* Nous pouvons bien sur ajouter un intercept $\beta_0$ à la fonction $f$. 

Or, un tel modèle ne tient pas compte de la périodicité de la série temporelle. Il est fort probable que la dépendance de la consommation au milieu de la nuit des $d$ dernières observations ne soit pas du tout la même que celle de la consommation du matin ou à midi. Cela veut dire que dans le modèle (2) il n'y a pas de vecteur $\beta$ qui donne des bonnes prédicitions quelque soit le moment de la journée (ou le jour de la semaine).
 

### Modèle avec périodicité
Pour prendre en compte une périodicité de durée $p$, nous pouvons opter pour un modèle qui cherche à exprimer la charge $x_{t+1}$ à l'instant $t+1$ en fonction des derniers instants modulo la période $p$. Par exemple, si la période $p$ est un jour et s'il est 13h30, nous allons alors chercher à exprimer $x_{t+1}$ en fonction de la charge observée les $d$ derniers jours toujours à 13h30. Si nous faisons l'hypothèse que la consommation dépend des $d$ dernières consommations à la même heure quelque soit l'heure, alors il convient d'utiliser le même $\beta$. Autrement dit, nous considérons un modèle de la forme
\begin{equation*}
x_{t+1} = f(x_{t+1-dp}, x_{t+1-(d-1)p}, ..., x_{t+1-p}) + \varepsilon_{t+1}.
\tag{3}
\end{equation*}
Dans ce modèle, il est possible de considérer n'importe quelle  périodicité (journalière ou hebdomadaire, par exemple). Remarquons que $p$ correspond au nombre de points d'observations dans une période, donc p.ex. $p=6*24=144$ pour une période d'un jour, ou bien $p=6*24*7=1008$ pour une période d'une semaine.

![Série temporelle](img/timeSeries3.jpeg)


Avec ce modèle, il sera également possible de prédire, au lieu qu'un seul point,  un nombre $\ell$ de points, en supposant $\ell\leq p$ (nous allons très souvent prendre $\ell=p$).
Il suffit en effet de poser 
\begin{equation*}
v_{t+1} = \begin{pmatrix}x_{t+1}\\ \vdots \\ x_{t+\ell} \end{pmatrix}
\quad \text{et} \quad 
v_{t-j} = \begin{pmatrix}x_{t+1 -(j+1)p}\\ \vdots \\ x_{t+\ell-(j+1)p} \end{pmatrix}
\quad \text{pour} \quad
0\leq j \leq d-1
\end{equation*}
et nous obtenons
$$v_{t+1} = \beta_1 v_{t-d+1}  + \ldots + \beta_d v_t + \tilde{\varepsilon}_{t+1}$$

où $\tilde{\varepsilon}_{t+1} = \begin{pmatrix}\varepsilon_{t+1}\\ \vdots \\ \varepsilon_{t+\ell} \end{pmatrix}$.
 


#### Tableau de données d'entrainement $X$

Il est clair que les observations les plus récentes sont les données les plus pertinentes pour entrainer le modèle, c'est-à-dire pour estimer $\beta$. On peut alors considérer comme vecteur $\mathbf y$ les $\ell$ valeurs observées :
$$\mathbf y = (x_{t-\ell+1},..., x_{t-1}, x_t)^\top,$$
où $t$ est l'instant actuel et donc la dernière valeur observée.

Maintenant il faut construire la matrice $X$ correspondante. Pour chaque entrée de $\mathbf y$ on choisit les observations qui ont "généré" cette observation selon le modèle en (3). 

Ainsi, la dernière ligne de $X$, qui correspond à l'observations $x_{t}$, est le vecteur
$$x_{t -dp}, x_{t-(d-1)p}, x_{t-(d-2)p},..., x_{t-p}
$$
Plus généralement, la $\ell-i$-ième ligne de $X$, associée à l'observations $x_{t-i+1}$, est le vecteur
$$x_{t-i -dp}, x_{t-i-(d-1)p }, x_{t-i-(d-2)p },..., x_{t-i-p}$$
 


![alternative text](img/timeSeries5bis.jpeg)

### Exercice 7

Mettre en oeuvre un prédicteur linéaire pour prédire le jour $k+1$ à partir des $d$ jours précedents.

- À l'aide de la fonction `LinearRegression` du module `sklearn.linear_model` faire une régression linéaire de la variable d'indice $k$ (qu'on renommera en `y_train`) sur les variables d'indice $k-1,\dots k-d$ (qu'on renommera en `x_train`) avec le jeu `df_day`. 
- Ensuite, grâce à cette relation apprise, prédire la variable d'indice $k+1$ (renommée `y_test`) à partir des $d$ variables immédiatement précédentes(renommées `x_test`).
- Comparer avec l'approche naïve, en affichant l'erreur quadratique et l'erreur MAPE au temps estimé $k+1$

### Réponse

In [ ]:
d = 4 # profondeur de l'historique (en jours ou bien en semaines)
k = 80 # date limite de l'historique (compris entre d+1 et 648)
# pour retrouver l'indice sous forme de paire (semaine observée, jour de la semaine)
# on pourra utiliser l'appel df_day_index[k]

y_train = df_day.loc[df_day_index[k]]
x_train = df_day.loc[df_day_index[k-d:k]]

y_test = df_day.loc[df_day_index[k+1]]
x_test = df_day.loc[df_day_index[k-d+1:k+1]]

In [ ]:
df_day.loc[df_day_index[k]]

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from scipy.stats import pearsonr 

# On prépare le modèle
lr = LinearRegression()

# On vérifie comment doivent être passées les données
lr.fit?

In [ ]:
# Les x_train et x_test doivent etre transposés
print(x_train.shape)
print(x_test.shape)

In [ ]:
x_train = x_train.T
x_test = x_test.T

In [ ]:
# On entraine le modèle
lr.fit(x_train, y_train)

In [ ]:
# On prédit et on calcule l'erreur commise
pred_day = lr.predict(x_test)

RMSE_linear_tplus1 = mean_squared_error(y_test,pred_day, squared=False)
MAPE_linear_tplus1 = mean_absolute_percentage_error(y_test,pred_day)

print(RMSE_linear_tplus1)
print(MAPE_linear_tplus1)

print(RMSE_lazy_day[k+1])
print(MAPE_lazy_day[k+1])

print(RMSE_lazy_week[k+1])
print(MAPE_lazy_week[k+1])

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(0,144),y_test, label="true value")
ax.plot(range(0,144),pred_day, label="linear regression prediction")
ax.plot(range(0,144),lazy_day.to_numpy(), label="lazy prediction")
ax.legend()

### Exercice 8

Mettre en oeuvre un prédicteur linéaire pour prédire la semaine qui contient le jour $k+1$ à partir des $d$ semaines précedentes.


### Réponse

In [ ]:
df_day_index[k+1][0]

In [ ]:
d=4
 
# Indices de semaine et de jour de la semaine précedente du jour k+1
t_week = df_day_index[k+1][0]-1
t_day = df_day_index[k+1][1]

y_train = df_week.loc[t_week]
x_train = df_week.loc[t_week-d:t_week-1].T

y_test = df_week.loc[t_week+1]
x_test = df_week.loc[t_week-d+1:t_week].T

lr = LinearRegression()
lr.fit(x_train, y_train)

pred_week = lr.predict(x_test)

pred_week = np.array(pred_week)
y_test = np.array(y_test)

pred_week_by_day = pred_week.reshape((-1,144), order='C')
y_test_by_day = y_test.reshape((-1,144), order='C')

pred_week_day = pred_week_by_day[t_day]
y_test_day = y_test_by_day[t_day]

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(0,144),y_test_day, label="true value")
ax.plot(range(0,144),pred_week_day, label="linear regression prediction by week")
ax.plot(range(0,144),pred_day, label="linear regression prediction by day")
ax.plot(range(0,144),lazy_day.to_numpy(), label="lazy prediction")
ax.legend()

### Exercice 9

Comparer les erreurs RMSE et MAPE pour des 4 prédicteurs sur l'ensemble des données.

### Réponse

In [ ]:
# Comparer sur l'ensemble des données RMSE et MAPE
pred_day_time = np.arange(d, df_day.shape[0]-1)

RMSE_linear_day = []
MAPE_linear_day = []
for t_ in pred_day_time:
    y_train = df_day.loc[df_day_index[t_]].to_numpy()
    x_train = df_day.loc[df_day_index[t_-d:t_]].to_numpy().T

    y_test = df_day.loc[df_day_index[t_+1]].to_numpy()
    x_test = df_day.loc[df_day_index[t_-d+1:t_+1]].to_numpy().T
    
    lr = LinearRegression()
    lr.fit(x_train, y_train)
    
    pred = lr.predict(x_test)

    RMSE_linear_day.append(mean_squared_error(y_test, pred, squared=False)) 
    MAPE_linear_day.append(mean_absolute_percentage_error(y_test, pred))
    
RMSE_linear_day = np.array(RMSE_linear_day)
MAPE_linear_day = np.array(MAPE_linear_day)

In [ ]:
# profondeur de l'historique EN SEMAINES
d =4

# Comparer sur l'ensemble des données RMSE et MAPE
week_range = np.arange(d+1, df_week.shape[0]-2) #on ne prend pas les lignes avec données manquantes

# On va prédire de la semaine d+2 à la semaine df_week.shape[0]-2

RMSE_linear_week = []
MAPE_linear_week = []

for w_ in week_range: 
    y_train = df_week.loc[w_].to_numpy()
    x_train = df_week.loc[w_-d:w_-1].to_numpy().T

    y_test = df_week.loc[w_+1].to_numpy()
    x_test = df_week.loc[w_-d+1:w_].to_numpy().T
    
    lr = LinearRegression()
    lr.fit(x_train, y_train)
    
    pred = lr.predict(x_test)
    
    pred = np.array(pred)
    y_test = np.array(y_test)
    
    pred_by_day = pred.reshape((-1,144), order='C')
    y_test_by_day = y_test.reshape((-1,144), order='C')
    
    for y_t,y_p in zip(y_test_by_day, pred_by_day) :
        RMSE_linear_week.append(mean_squared_error(y_t, y_p, squared=False)) 
        MAPE_linear_week.append(mean_absolute_percentage_error(y_p, y_t))
    
RMSE_linear_week = np.array(RMSE_linear_week)
MAPE_linear_week = np.array(MAPE_linear_week)

In [ ]:
# convertir en t pour avoir le range pour les plots
#first_week_index = df_week.loc[d+2:df_week.shape[0]-2].stack([0]).index[0]
#last_week_index = df_week.loc[d+2:df_week.shape[0]-2].stack([0]).index[-1]

#week_pred_range = np.arange(df_day_index.get_indexer([first_week_index]), df_day_index.get_indexer([last_week_index])+1)

first_week_index = df_day_index.get_loc((d+2,0))
last_week_index = df_day_index.get_loc((df_week.shape[0]-2,6))

week_pred_range = np.arange(first_week_index, last_week_index+1)

print(week_pred_range[0])
print(week_pred_range[-1])

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=1, nrows=2, sharex=True, layout="tight")
ax1.plot(RMSE_lazy_day, label="RMSE (day)", linewidth=1)
ax1.plot(RMSE_lazy_week, label="RMSE (week)", linewidth=1)
ax1.plot(pred_day_time, RMSE_linear_day, label="RMSE (linear day)", linewidth=1)
ax1.plot(week_pred_range, RMSE_linear_week, label="RMSE (linear week)", linewidth=1)
ax1.set_title("RMSE")
ax2.plot(MAPE_lazy_day, label="MAPE (day)", linewidth=1)
ax2.plot(MAPE_lazy_week, label="MAPE (week)", linewidth=1)
ax2.plot(pred_day_time,MAPE_linear_day, label="MAPE (linear day)", linewidth=1)
ax2.plot(week_pred_range,MAPE_linear_week, label="MAPE (linear week)", linewidth=1)
ax2.set_title("MAPE")
for ax in (ax1, ax2): ax.legend()
plt.show()

In [ ]:
print(RMSE_linear_day.mean())
print(RMSE_linear_week.mean())

print(MAPE_linear_day.mean())
print(MAPE_linear_week.mean())

Cette approche n'utilise qu'une petite partie des données. Si on suppose que le modèle est relativement stable sur une plus longue période, on peut bien évidemment   ajuster $\beta$ sur plus de données. Pour cela, on considère pour l'entrainement un vecteur $\mathbf y$ plus long
$$\mathbf y = (x_t,x_{t-1}...,x_{t-m})^\top,$$
avec $m$ plus grand que $\ell$ (qui était le nombre de valeurs à prédire).
La matrice $X$ est construite de la même façon que précédemment.


## Forêts aléatoires 

Dans cette partie, on va encore chercher le lien entre  la courbe d'indice $k$ et les courbes de charges précedentes $k-1,\dots,k-d$. Mais cette fois, le lien entre ces variables sera appris par des arbres de décision ; plus précisémment, par des forêts aléatoires. 

### Exercice 10

Vous allez à présent utiliser des forêts aléatoires via le module `RandomForestRegressor` de `sklearn.ensemble` pour apprendre un modèle prédictif pour le même problème et les mêmes données qu'avant. 

Explorez les paramètres de la fonction `RandomForestRegressor`.

Procédez comme précédemment : 
- apprendre pour une semaine fixée $k$, avec un historique $d$, puis prédire la semaine $k+1$. Calculez différentes erreurs pour cette période.  
- calculer les erreurs moyennes sur toutes les périodes disponibles.

### Réponse

In [ ]:
# Modelisation Random Forest

from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor

# Numero colonne du debut la periode d'apprentissage ==> K+1 à prévoir
k = 60

# Nombre de periodes passees : historique utilisé
d = 4

# Creation du modele
rf = RandomForestRegressor(
    n_estimators=100, # Nombre d'échantillons bootstrap et d'arbres
    max_features=0.2, #  round(max_features * n_features) features are considered at each split (The default of 1.0 is equivalent to bagged trees)
    max_depth=10,      # profondeur max de l'arbre
    random_state=2,    # on règle une graine pour avoir tous les mêmes résultats
    oob_score=True # Whether to use out-of-bag samples to estimate the generalization score. 
)

y_train = df_day.loc[df_day_index[k]]
x_train = df_day.loc[df_day_index[k-d:k]].T

y_test = df_day.loc[df_day_index[k+1]]
x_test = df_day.loc[df_day_index[k-d+1:k+1]].T

nobs = y_test.shape[0]

# On ajuste le modèle
rf.fit(x_train, y_train)
# On prédit la période suivante 
pred = rf.predict(x_test)

print(pred.shape[0])

rmse = mean_squared_error(y_test, pred, squared=False)
print("RMSE: %.4f" % rmse)
map1 = mean_absolute_percentage_error(pred, y_test)
print("MAPE: %.4f" % map1)
cor = pearsonr(y_test, pred)
print("Correlation observation/prevue: %.4f" % cor[0])

fig = plt.figure(figsize=(15, 3))
x = np.linspace(1, nobs, nobs)
plt.plot(x, y_test, label="observation", lw=2)
plt.plot(x, pred, label="prévision", lw=2)
plt.title('Courbes de charge', fontsize=18)
plt.legend(fontsize=14, loc='upper left')
plt.show()

Maintenant qu'on a fait cela pour une période donnée, on systématise sur l'ensemble des périodes disponibles et on calcule les erreurs moyennes correspondantes. 

In [ ]:
? RandomForestRegressor

In [ ]:
# Modelisation RF
# Systematique
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor

# Creation du modele
rf = RandomForestRegressor(n_estimators=50, max_features="sqrt", 
                           max_depth=3)

RMSE_forest_day = []
MAPE_forest_day = []
for t_ in pred_day_time:
    y_train = df_day.loc[df_day_index[t_]].to_numpy()
    x_train = df_day.loc[df_day_index[t_-d:t_]].to_numpy().T

    y_test = df_day.loc[df_day_index[t_+1]].to_numpy()
    x_test = df_day.loc[df_day_index[t_-d+1:t_+1]].to_numpy().T
    
    rf.fit(x_train, y_train)
    
    pred = rf.predict(x_test)
    
    RMSE_forest_day.append(mean_squared_error(y_test, pred, squared=False)) 
    MAPE_forest_day.append(mean_absolute_percentage_error(pred, y_test))
    
RMSE_forest_day = np.array(RMSE_forest_day)
MAPE_forest_day = np.array(MAPE_forest_day)



In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=1, nrows=2, sharex=True, layout="tight")

ax1.plot(pred_day_time, RMSE_linear_day, label="RMSE (linear day)", linewidth=1)
ax1.plot(pred_day_time, RMSE_forest_day, label="RMSE (forest day)", linewidth=1)
ax1.set_title("RMSE")
ax2.plot(pred_day_time,MAPE_linear_day, label="MAPE (linear day)", linewidth=1)
ax2.plot(pred_day_time,MAPE_forest_day, label="MAPE (forest day)", linewidth=1)
ax2.set_title("MAPE")
for ax in (ax1, ax2): ax.legend()
plt.show()

In [ ]:
print(RMSE_linear_day.mean())
print(RMSE_forest_day.mean())

print(MAPE_linear_day.mean())
print(MAPE_forest_day.mean())

In [ ]:

RMSE_forest_week = []
MAPE_forest_week = []

for w_ in week_range: 
    y_train = df_week.loc[w_].to_numpy()
    x_train = df_week.loc[w_-d:w_-1].to_numpy().T

    y_test = df_week.loc[w_+1].to_numpy()
    x_test = df_week.loc[w_-d+1:w_].to_numpy().T

    rf.fit(x_train, y_train)
    
    pred = rf.predict(x_test)
    
    pred = np.array(pred)
    y_test = np.array(y_test)
    
    pred_by_day = pred.reshape((-1,144), order='C')
    y_test_by_day = y_test.reshape((-1,144), order='C')
    
    for y_t,y_p in zip(y_test_by_day, pred_by_day) :
        RMSE_forest_week.append(mean_squared_error(y_t, y_p, squared=False)) 
        MAPE_forest_week.append(mean_absolute_percentage_error(y_p, y_t))
    
RMSE_forest_week = np.array(RMSE_forest_week)
MAPE_forest_week = np.array(MAPE_forest_week)

In [ ]:
first = 100 # has to be at least week_pred_range[0]
last = 200 # at last week_pred_range[-1]

first_w = np.where(week_pred_range == first)[0][0]
last_w = np.where(week_pred_range == last)[0][0]

fig, (ax1, ax2) = plt.subplots(ncols=1, nrows=2, sharex=True, layout="tight")
ax1.plot(np.arange(first, last),RMSE_lazy_week[first:last], label="RMSE (week)", linewidth=1)
ax1.plot(week_pred_range[first_w:last_w], RMSE_linear_week[first_w:last_w], label="RMSE (linear week)", linewidth=1)
ax1.plot(week_pred_range[first_w:last_w], RMSE_forest_week[first_w:last_w], label="RMSE (forest week)", linewidth=1)
ax1.set_title("RMSE")
ax2.plot(np.arange(first, last),MAPE_lazy_week[first:last], label="MAPE (week)", linewidth=1)
ax2.plot(week_pred_range[first_w:last_w],MAPE_linear_week[first_w:last_w], label="MAPE (linear week)", linewidth=1)
ax2.plot(week_pred_range[first_w:last_w],MAPE_forest_week[first_w:last_w], label="MAPE (forest week)", linewidth=1)
ax2.set_title("MAPE")
for ax in (ax1, ax2): ax.legend()
plt.show()

In [ ]:
print(RMSE_lazy_week.mean())
print(RMSE_linear_week.mean())
print(RMSE_forest_week.mean())

print(MAPE_lazy_week.mean())
print(MAPE_linear_week.mean())
print(MAPE_forest_week.mean())

## Gradient Boosting

Dans cette partie, à l'aide de la fonction ` GradientBoostingRegressor` du module ` sklearn.ensemble`,  vous allez utiliser le gradient boosting pour apprendre la relation entre une courbe de charge à une période $k$ fixée, et l'historique des $d$ courbes précédentes.

 


### Exercice 11

- Commencez par utiliser les paramètres par défaut de l'algorithme. 
- Entraînez le modèle pour une période $k$ fixée et faites la prédiction de la même façon que ci-dessus. En particulier, vous présenterez les mêmes graphiques et les mêmes calculs d'erreurs. 
- Choisissez les meilleurs paramètres de l'algorithme en utilisant la fonction `GridSearchCV`, pour essayer de diminuer les erreurs. 
- Enfin, évaluez la qualité de la méthode en utilisant toutes les valeurs de $k$ possibles (selon la longueur des données dont on dispose).

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
?GradientBoostingRegressor
gb = GradientBoostingRegressor()

### Réponse

In [ ]:
# Modelisation avec Gradient Boosting

# Choisir k numero de periode
# Apprendre le lien Pk et son passe
# Prevoir Pk+1

from sklearn.ensemble import GradientBoostingRegressor

# Numero colonne du debut la periode d'apprentissage ==> K+1 à prévoir
k = 60

# Nombre de periodes passees : historique utilisé
d = 4

# Creation du modele
gb = GradientBoostingRegressor()

y_train = df_day.loc[df_day_index[k]]
x_train = df_day.loc[df_day_index[k-d:k]].T

y_test = df_day.loc[df_day_index[k+1]]
x_test = df_day.loc[df_day_index[k-d+1:k+1]].T

nobs = y_test.shape[0]

# On ajuste le modèle
gb.fit(x_train, y_train)
# On prédit la période suivante 
pred = gb.predict(x_test)

print(pred.shape[0])

rmse = mean_squared_error(y_test, pred, squared=False)
print("RMSE: %.4f" % rmse)
map1 = mean_absolute_percentage_error(pred,y_test)
print("MAPE: %.4f" % map1)
cor = pearsonr(y_test, pred)
print("Correlation observation/prevue: %.4f" % cor[0])

fig = plt.figure(figsize=(15, 3))
x = np.linspace(1, nobs, nobs)
plt.plot(x, y_test, label="observation", lw=2)
plt.plot(x, pred, label="prévision", lw=2)
plt.title('Courbes de charge', fontsize=18)
plt.legend(fontsize=14, loc='upper left')
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'learning_rate': [0.001, 0.01, 0.05, 0.1],
    'n_estimators': [100, 200],
    'subsample' : [0.25,0.5,0.75,1],
    'max_depth' : [2, 3, 4]
}

gbm = GridSearchCV(gb, param_grid, cv=10)

# On ajuste le modèle
gbm.fit(x_train, y_train) # fit sur tout les jeux de paramètres

In [ ]:
print('Les meilleurs paramètres sur la grille sont ', gbm.best_params_)

In [ ]:
gbm.best_params_['learning_rate']

In [ ]:
%%time

# Creation du modele avec les paramètres optimaux
gb = GradientBoostingRegressor(
    learning_rate=gbm.best_params_['learning_rate'], 
    max_depth=gbm.best_params_['max_depth'],
    n_estimators=gbm.best_params_['n_estimators'],
    subsample=gbm.best_params_['subsample']
)

RMSE_gb_day = []
MAPE_gb_day = []
for t_ in pred_day_time:
    y_train = df_day.loc[df_day_index[t_]]
    x_train = df_day.loc[df_day_index[t_-d:t_]].T

    y_test = df_day.loc[df_day_index[t_+1]]
    x_test = df_day.loc[df_day_index[t_-d+1:t_+1]].T
    
    gb.fit(x_train, y_train) # fit sur tout les jeux de paramètres
    
    pred = gb.predict(x_test) # predit avec le meilleur jeu de params
    
    RMSE_gb_day.append(mean_squared_error(y_test, pred, squared=False)) 
    MAPE_gb_day.append(mean_absolute_percentage_error(pred, y_test))
    
RMSE_gb_day = np.array(RMSE_gb_day)
MAPE_gb_day = np.array(MAPE_gb_day)




In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=1, nrows=2, sharex=True, layout="tight")

ax1.plot(pred_day_time, RMSE_linear_day, label="RMSE (linear day)", linewidth=1)
ax1.plot(pred_day_time, RMSE_forest_day, label="RMSE (forest day)", linewidth=1)
ax1.plot(pred_day_time, RMSE_gb_day, label="RMSE (gb day)", linewidth=1)
ax1.set_title("RMSE")
ax2.plot(pred_day_time,MAPE_linear_day, label="MAPE (linear day)", linewidth=1)
ax2.plot(pred_day_time,MAPE_forest_day, label="MAPE (forest day)", linewidth=1)
ax2.plot(pred_day_time, MAPE_gb_day, label="MAPE (gb day)", linewidth=1)
ax2.set_title("MAPE")
for ax in (ax1, ax2): ax.legend()
plt.show()

In [ ]:
print(RMSE_linear_day.mean())
print(RMSE_forest_day.mean())
print(RMSE_gb_day.mean())

print(MAPE_linear_day.mean())
print(MAPE_forest_day.mean())
print(MAPE_gb_day.mean())